In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Burundi_Code_2017_penal.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
# data

In [9]:
len(data)

128

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
# print(docs[200])
# print("================================================")
print("Total number of documents: ",len(docs))

Total number of documents:  436


In [25]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207, -0.030764883384108543]

In [27]:

vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("Quelles sont les différentes catégories d'infractions selon le Code pénal burundais ?")

In [33]:
len(retrieved_docs)

10

In [34]:
print(retrieved_docs[5].page_content)

www.droit‐afrique.com	 Burundi
	 	
	
Code	pénal	2017	 20
2°	Lorsque	l’action	comprend	des	faits	qui,	constituant	des	inf ractions	distinctes,	sont	
unis	entre	eux	comme	procédant	d ’une	intention	délictuelle	unique	ou	comme	étant	les	
uns	des	circonstances	aggravantes	des	autres.	
Dans	l’un	et	l’autre	cas,	la	peine	la	plus	forte	est	seule	prononcée.	
Art.114.‐	Il	y	a	concours	réel	lorsque	les	faits,	distincts	au	point	d e	vue	matériel,	se	sont	
succédés	et	ont	constitué	des	infractions	distinctes.	Dans	ce	c as	il	est	prononcé	des	
peines	pour	chaque	infraction	et	les	peines	prononcées	sont	cumulées	sous	réserves	des	
dispositions	suivantes	:	
1°	La	servitude	pénale	à	perpétuité	absorbe	de	droit	toute	peine	privative	de	liberté	;	
2°	Le	total	des	peines	cumulées	de	servitude	pénale	à	temps	et	 des	amendes	ne	peut	
dépasser	le	double	du	maximum	des	peines	les	plus	fortes	prévue s	par	l’une	ou	l’autre	
infraction	retenue	contre	le	condamné	;


In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite",temperature=0.7, max_tokens=500)

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Vous êtes un assistant spécialisé dans la réponse aux questions concernant le Code pénal du Burundi."
    "Utilisez uniquement les extraits de contexte fournis pour formuler votre réponse "
    "Si la réponse n’est pas présente dans le contexte, indiquez que vous ne savez pas "
    "Répondez de manière claire, précise et en trois phrases maximum "
    "Réponse flexible dans ta manière de parler.."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [55]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "Le Code pénal aborde-t-il la question de la sorcellerie, et si oui, quelles sont les peines associées ?"})
print(response["answer"])

Selon le Code pénal du Burundi, les infractions sont classées en trois catégories selon leur gravité. Ces catégories sont les crimes, les délits et les contraventions. Les crimes sont les infractions les plus graves, suivis des délits et enfin des contraventions, qui sont les moins graves.
